In [ ]:
# Import packages
import pandas as pd
from os import environ
from auth import Auth
from workspace import Workspace
from dataset import Dataset

# Tenant/app settings
TENANT_ID = environ.get('TENANT_ID', '')
CLIENT_ID = environ.get('CLIENT_ID', '')
CLIENT_SECRET = environ.get('CLIENT_SECRET', '')

In [ ]:
# Authentication (get bearer token)
auth = Auth(TENANT_ID, CLIENT_ID, CLIENT_SECRET)
token = auth.get_token()

In [ ]:
# Initializing objects
workspace = Workspace(token)
dataset = Dataset(token)

In [ ]:
# Term to search on workspace name
workspace_to_search = 'workspace'

# Filter workspaces that contain search word or is Dataflows workspace
# Saves to an Excel (.xslx) file
workspaces = workspace.list_workspace(filters=f"contains(name,'{workspace_to_search}')%20or%20name%20eq%20'Dataflows'")
workspaces_list = workspaces.get('content', [])

# See content
n = 2
print(f'Found {len(workspaces_list)} workspaces...\nPrinting first {n}:')
pd.DataFrame(workspaces_list).head(n)

In [ ]:
user = 'user@domain.com'
df = workspace.batch_update_user(user=user, workspaces_list=workspaces_list)

In [ ]:
# Success
df.loc[df.status == 'Success']

In [ ]:
# Errors
df.loc[df.status != 'Success']